In [28]:
import pandas as pd
from dash import Dash, html, dcc, callback, Output, Input
import censusgeocode as cg
import plotly.figure_factory as ff
from urllib.request import urlopen
import json
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
from sklearn.linear_model import LinearRegression
from Partisian import get_state_part_score
from Life import get_age_life_fig, get_state_age_life_exp
from sklearn.neighbors import KNeighborsClassifier
from plotly.subplots import make_subplots
import math
from LifeData import LifeData
import Partisian
from sklearn.linear_model import LinearRegression

In [2]:
import base64
import gzip

In [ ]:
response_document= "H4sIANRaXmQAAwMAAAAAAAAAAAA=\n"
gzip.decompress(base64.b64decode(response_document))

In [ ]:
df = pd.read_csv(r'./data/Life/LifeExp/StateLifeExp2020CDC.csv')
fig = go.Figure(data=go.Choropleth(
    locations = df["STATE"],
    z = df["RATE"],
    locationmode = 'USA-states',
    colorscale='Inferno',
    colorbar_title="Life Expectancy"    
))
fig.update_layout(
    title_text ="2020 Life Expectancy - State Wide",
    geo_scope='usa',
)

In [ ]:
df

In [ ]:
df = pd.read_csv(r'./data/Life/US_A.CSV')
df["Tract ID"] = df["Tract ID"].astype(str)
df["FIPS"] = df["Tract ID"].str[:-6].str.zfill(5)
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

In [ ]:
#fig = px.choropleth(df, geojson=counties, locations='FIPS', color='e(0)',
#                           color_continuous_scale="Viridis",
#                           range_color=(0, 12),
#                           scope="usa",
#                           labels={'e(0)':'Life Expectancy'}
#                          )
#fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
#fig.show()

In [ ]:
income_life_exp = pd.read_csv(r"data\Life\health_ineq_online_table_2.csv")

In [ ]:
income_life_exp

In [ ]:
income_life_exp[(income_life_exp["gnd"]=="F") & (income_life_exp["year"]==2001)]["pctile"],

In [ ]:
year = 2014
df = income_life_exp[income_life_exp["year"]==2014]
fig = make_subplots(rows =2, cols =1)
fig.add_trace(go.Scatter(x = df[(df["gnd"]=="M")]["pctile"], y = df[(df["gnd"]=="M")]["le_agg"] ),
              row=1, col=1)
fig.add_trace(go.Scatter(x = df[(df["gnd"]=="F")]["pctile"], y = df[(df["gnd"]=="F")]["le_agg"] ),
              row=1, col=1)


fig.add_trace(go.Bar(x=["Male Female Diff"], y = [df[df.gnd=="F"]["le_agg"].mean() - df[df.gnd=="M"]["le_agg"].mean()]),
              row=2, col=1)
fig.add_trace(go.Bar(x=["Top Bottom"], y = [df[df.pctile>90]["le_agg"].mean() - df[df.pctile<10]["le_agg"].mean()]),
              row=2, col=1)






fig.show()

In [ ]:
income_life_exp[income_life_exp.year < 2014].groupby(by=["year","gnd"]).mean().reset_index()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=df["e(0)"]))

In [ ]:
fig = px.histogram(df, x="e(0)", color="STATE2KX")
fig.update_layout(barmode='overlay')
fig.show()

In [ ]:
#https://www.ncsl.org/about-state-legislatures/state-partisan-composition


In [ ]:
state_score = get_state_part_score(2019)

In [ ]:
df = pd.read_csv(r"data\Life\StateLevel2018.csv")
df = df[df["YEAR"] < 2019][["YEAR","STATE","RATE"]]
life_v_part = df.merge(state_score, left_on="STATE", right_on="State")
life_v_part = life_v_part.drop(columns=["State"])
life_v_part = life_v_part.dropna()

In [ ]:
fig = px.scatter(life_v_part, x="Part_Score", y="RATE", color="Part_Score", color_continuous_scale=px.colors.sequential.Bluered, trendline="ols")
fig.show()

In [ ]:
reg = LinearRegression().fit(life_v_part["Part_Score"].values.reshape(-1, 1), life_v_part["RATE"])
X = np.linspace(-1,1)
y = reg.predict(X.reshape(-1,1))


In [ ]:


fig = go.Figure()
fig.add_trace(go.Scatter(x=X, y=y, mode="lines", marker=dict(color="orange")))
fig.add_trace(go.Scatter(x=life_v_part["Part_Score"],
    y=life_v_part["RATE"],
    mode="markers",
    marker=dict(
        size=8,
        color=life_v_part["Part_Score"], #set color equal to a variable
        colorscale=px.colors.sequential.Bluered, # one of plotly colorscales
        showscale=True,),
    text=life_v_part['STATE']          
    ))
fig.show()

In [ ]:
fig = px.histogram(life_v_part, x="RATE", color="category")
fig.update_layout(barmode='overlay')
fig.show()

In [ ]:
ca = pd.read_excel(r"data\Life\State\CA1.xlsx", header=2,skipfooter=1)
ca.rename(columns={ ca.columns[0]: "age" }, inplace = True)

fl = pd.read_excel(r"data\Life\State\FL1.xlsx", header=2,skipfooter=1)
fl.rename(columns={ fl.columns[0]: "age" }, inplace = True)

In [ ]:

def get_part_cat(score):
    if score<-.2:
        return "Left"
    elif score<.3:
        return "Center"
    else:
        return "Right"
state_score = state_score.dropna()
state_score["category"] = state_score["Part_Score"].apply(lambda x:get_part_cat(x))


In [ ]:
df = pd.read_excel(r"./data/Life/State/NY1.xlsx", header=2,skipfooter=1)
df.rename(columns={ df.columns[0]: "age" }, inplace = True)
age_categories = list(df["age"].values)

all_states = []
left = []
center =[]
right =[]

for index, state in state_score.iterrows():
    state_name = state["State"]
    df = pd.read_excel(r"./data/Life/State/"+state_name+"1.xlsx", header=2,skipfooter=1)
    df.rename(columns={ df.columns[0]: "age" }, inplace = True)
    df["age_1"] = df["age"].str.replace(" ","–").str.split("–").str[0].astype(int)
    df["state"] = state_name
    all_states.append(df)
    if state["category"] == "Left":
        left.append(df)
    if state["category"] == "Center":
        center.append(df)
    if state["category"] == "Right":
        right.append(df)
left = pd.concat(left)
center = pd.concat(center)
right = pd.concat(right)
all_states = pd.concat(all_states)


left = left.groupby(by=["age"])[["qx","lx","dx","Lx","Tx","ex", "age_1"]].mean().reset_index()
left["age"] = pd.Categorical(left["age"], categories=age_categories)
left.sort_values(by="age", inplace=True)

center = center.groupby(by=["age"])[["qx","lx","dx","Lx","Tx","ex", "age_1"]].mean().reset_index()
center["age"] = pd.Categorical(center["age"], categories=age_categories)
center.sort_values(by="age", inplace=True)

right = right.groupby(by=["age"])[["qx","lx","dx","Lx","Tx","ex", "age_1"]].mean().reset_index()
right["age"] = pd.Categorical(right["age"], categories=age_categories)
right.sort_values(by="age", inplace=True)


all_states = state_score[["State","Part_Score"]].merge(all_states, left_on="State", right_on="state")


population = pd.read_csv("./data/Population/us-state-populations.csv").rename({"pop_2014":"pop"}, axis=1)
all_states = all_states.merge(population, left_on="State",right_on="code")


In [ ]:

center["left_right_diff"] = left["lx"] - right["lx"]
fig = go.Figure()
fig.add_trace(go.Scatter(x=left["age"], y=left["lx"],  name="Left"))
fig.add_trace(go.Scatter(x=center["age"], y=center["lx"],  name="Center"))
fig.add_trace(go.Scatter(x=right["age"], y=right["lx"], name="Right"))
fig.add_trace(go.Scatter(x=center["age"], y=center["left_right_diff"],  name="Diff"))
#fig.update_yaxes(type="log")
fig.show()

In [ ]:
all_states

In [ ]:
fig = go.Figure(
    data=[go.Bar(x=["Left"], y=left[left["age_1"] == 0.0]["ex"],name="Left"),
          go.Bar(x=["Center"], y=center[center["age_1"] == 0.0]["ex"],name="Center"),
          go.Bar(x=["Right"], y=right[right["age_1"] == 0.0]["ex"],name="Right"),
          go.Bar(x=["Diff"],y=left[left["age_1"] ==  0.0]["ex"] - right[right["age_1"] ==  0.0]["ex"],name="Diff"),
          
          
          ],
    layout=go.Layout(
        xaxis={'title':f"Age : {0}",},
        yaxis ={'range':[0,85], "autorange":False},
        updatemenus=[dict(type="buttons",
                          buttons=[dict(label="Play",
                                        method="animate",
                                        args=[None])])]),
    frames=[go.Frame(
        data=[go.Bar(x=["Left"],y=left[left["age_1"] == float(k)]["ex"],name="Left"),
              go.Bar(x=["Center"],y=center[center["age_1"] == float(k)]["ex"],name="Center"),
              go.Bar(x=["Right"],y=right[right["age_1"] == float(k)]["ex"],name="Right"),
              go.Bar(x=["Diff"],y=left[left["age_1"] == float(k)]["ex"] - right[right["age_1"] == float(k)]["ex"],name="Diff"),
              
              ], 
        layout=go.Layout(
              xaxis={'title':f"Age : {k}",},
              yaxis ={'range':[0,85], "autorange":False}
              ))

        for k in range(101)]
)

fig.show()

In [ ]:

from Life import get_state_age_life_exp, get_age_categories

all_states = get_state_age_life_exp()
age_categories = get_age_categories()
state_score = get_state_part_score(2019)
population = pd.read_csv("./data/Population/us-state-populations.csv").rename({"pop_2014":"pop"}, axis=1)

frames = []
coef = []
for k in range(101):
      age_df = all_states[all_states["age_1"]==float(k)]
      
      reg = LinearRegression().fit(age_df["Part_Score"].values.reshape(-1, 1), age_df["lx"])
      X = np.linspace(-1,1)
      y = reg.predict(X.reshape(-1,1))
      #coef.append({"y":reg.coef_, "age":k})
      coef.append(float(reg.coef_))
  
      data = [go.Scatter(x=age_df["Part_Score"], 
                         y=age_df["lx"], 
                         mode="markers", hovertext=age_df["State"],
                         marker=dict(color = age_df["Part_Score"], size=age_df["pop"]/500000, colorscale='bluered')),
              go.Scatter(x=X, 
                         y=y, 
                         mode="lines")
              ]
      
      if k < 50:
        highest = 100000
        lowest = 0
      else:
        highest=100000
        lowest=0
      #highest = max(0, age_df["lx"].max())
      #lowest = max(0, age_df["lx"].min())
      
      lowest = max(0, age_df["lx"].max()-20000)
      highest =max(age_df["lx"].max() + 1000, 20000)
      #print(f"High: {highest} , Low : {lowest}, diff {highest-lowest}")
      layout=go.Layout(
              xaxis={'title':f"Age : {k}",},
              yaxis ={"autorange":False, "range":[lowest,highest+.0005] },
              )
      
      frames.append(go.Frame(data=data, layout=layout))
      
      
updatemenus = [
    dict(
        type='buttons',
        buttons=[dict(label="Play",
                      method="animate",
                      args=[None,
                        dict(frame=dict(duration=240, redraw=True),
                              transition=dict(
                                  duration=.5,
                                  easing="linear" # <<=====
                              ),
                              fromcurrent=True,
                              mode="immediate")])],
        direction="left",
        pad=dict(r=10, t=85),
        showactive=True, x=1.15, y=1, xanchor="right", yanchor="top")]


fig = go.Figure(
    data=frames[0].data,
    
    layout=frames[0].layout,
    frames=frames
)

fig.update_layout(
  updatemenus=updatemenus, 
  # sliders=sliders
)

fig.show()

In [ ]:
all_states

In [ ]:
coef
fig = go.Figure()
fig.add_trace(go.Scatter(x = [x for x in range(100)], y = coef))

In [ ]:
coef

In [ ]:
age_df = all_states[all_states["age_1"]==float(k)]
      
reg = LinearRegression().fit(age_df["Part_Score"].values.reshape(-1, 1), age_df["qx"])
X = np.linspace(-1,1)
y = reg.predict(X.reshape(-1,1))

In [ ]:
reg.coef_

In [ ]:
layout=go.Layout(
        updatemenus=[dict(type="buttons",
                          buttons=[dict(label="Play",
                                        method="animate",
                                        args=[
                        dict(frame=dict(duration=30, redraw=True),
                              transition=dict(
                                  duration=0,
                                  easing="linear" # <<=====
                              ),
                              #easing="linear", # <<=====
                              fromcurrent=True,
                              mode="immediate")])])])

In [ ]:
[
                        dict(frame=dict(duration=30, redraw=True),
                              transition=dict(
                                  duration=0,
                                  easing="linear" # <<=====
                              ),
                              #easing="linear", # <<=====
                              fromcurrent=True,
                              mode="immediate")]

In [ ]:
layout

In [ ]:
fig = px.scatter(all_states, x="Part_Score",y="qx", animation_frame="age_1", color="Part_Score", color_continuous_scale=px.colors.sequential.Bluered, trendline="ols")
fig.show()

In [ ]:
https://ourworldindata.org/life-expectancy


In [ ]:

df

In [ ]:
df = get_state_age_life_exp()
#df = df.groupby(by=["category", "age_1"]).mean().reset_index()
df["percent"] = df["lx"]/100000
#df = df.sort_values(by=["category", "age_1"])

In [ ]:
fig = px.line(df, x = "age_1", y ="percent", color = "category")
fig.show()

In [ ]:
fig = px.line(df, x = "age", y ="percent", color = "State")
fig.show()

In [ ]:
df = pd.read_csv("./data/Life/health_ineq_online_table_3.csv")
df.columns
df = df[['stateabbrv', 'le_agg_q1_F','le_agg_q2_F', 'le_agg_q3_F', 'le_agg_q4_F', 'le_agg_q1_M', 'le_agg_q2_M', 'le_agg_q3_M', 'le_agg_q4_M']]
df.head(3)

In [ ]:
df[df.le_agg_q4_F == df.le_agg_q4_F.min()]
#df[df.le_agg_q4_F == df.le_agg_q4_F.max()]

In [ ]:
selected_states = []

for quartile in range(1,5):

    selected_states.append(df[df[f"le_agg_q{quartile}_F"] == df[f"le_agg_q{quartile}_F"].min()]["stateabbrv"].values[0])
    #selected_states.append(df[df[f"le_agg_q{quartile}_M"] == df[f"le_agg_q{quartile}_M"].min()]["stateabbrv"].values[0])
    
    selected_states.append(df[df[f"le_agg_q{quartile}_F"] == df[f"le_agg_q{quartile}_F"].max()]["stateabbrv"].values[0])
    #selected_states.append(df[df[f"le_agg_q{quartile}_M"] == df[f"le_agg_q{quartile}_M"].max()]["stateabbrv"].values[0])
    
    
selected_states = list(set(selected_states))
#selected_states.remove("DC")

In [ ]:
selected_states

In [ ]:
df[df["stateabbrv"].isin(selected_states)]

In [ ]:
fig =go.Figure()
for state in selected_states:
    state_df = df[df["stateabbrv"] == state]
    columns = [f"le_agg_q{i}_F" for i in range(1,5)]
    fig.add_trace(go.Bar(
        x = columns,
        y = state_df[columns].values[0],
        name = state
        ))
fig.update_layout(barmode='group')
fig.update_yaxes(range=[70, 90])
fig.show()

In [ ]:
state_df = df[df["stateabbrv"] == "AL"]
state_df

In [ ]:
columns = [f"le_agg_q{i}_F" for i in range(1,5)]
state_df[columns].values[0]

In [ ]:
from LifeData import LifeData
life_data = LifeData()
    
df = life_data.get_single_indicator("Life Exp")
df = df.sort_values(by=["Rate"], ascending=False).reset_index(drop=True)
pivot_state =life_data.get_long_format()

In [29]:
import Partisian
from LifeData import LifeData
life_data = LifeData()
    
df = life_data.get_single_indicator("Life Exp")
df = df.sort_values(by=["Rate"], ascending=False).reset_index(drop=True)
pivot_state =life_data.get_long_format()

df = Partisian.get_state_part_score()
merged_df = pd.merge(df, pivot_state)
merged_df.sample(3)

Checking for NAs in 2020 ind ['Life Exp'] rate 0
Checking for NAs in 2019 ind ['Life Exp'] rate 0
Checking for NAs in 2018 ind ['Life Exp'] rate 0
Checking for NAs in 2020 ind ['Infant Mortality'] rate 0
Checking for NAs in 2019 ind ['Infant Mortality'] rate 0
Checking for NAs in 2018 ind ['Infant Mortality'] rate 0
Checking for NAs in 2017 ind ['Infant Mortality'] rate 0
Checking for NAs in 2016 ind ['Infant Mortality'] rate 0
Checking for NAs in 2015 ind ['Infant Mortality'] rate 0
Checking for NAs in 2014 ind ['Infant Mortality'] rate 0
Checking for NAs in 2005 ind ['Infant Mortality'] rate 0
Checking for NAs in 2021 ind ['Fertility Rate'] rate 0
Checking for NAs in 2020 ind ['Fertility Rate'] rate 0
Checking for NAs in 2019 ind ['Fertility Rate'] rate 0
Checking for NAs in 2018 ind ['Fertility Rate'] rate 0
Checking for NAs in 2017 ind ['Fertility Rate'] rate 0
Checking for NAs in 2016 ind ['Fertility Rate'] rate 0
Checking for NAs in 2015 ind ['Fertility Rate'] rate 0
Checking for

,State,Part_Score,Year,category,Fertility Rate,Infant Mortality,Life Exp,Low Birth Weight,Premature,Teen Birth,Unmarried Birth
17,VA,-0.133333,2024,Center,56.099998,5.76,77.599998,8.33,9.90,11.7,34.799999
28,MI,0.400000,2024,Right,55.400002,6.80,76.000000,9.22,10.60,12.2,40.299999
15,ME,-0.333333,2024,Left,49.900002,6.33,77.800003,7.29,9.37,7.8,37.200001


In [30]:
merged_df
merged_df[merged_df["FL"] == state]["Infant Mortality"]

KeyError: 'FL'

In [31]:
pivot_state.sample(3)

Indicator,State,Fertility Rate,Infant Mortality,Life Exp,Low Birth Weight,Premature,Teen Birth,Unmarried Birth
38,RI,48.299999,3.96,78.199997,7.92,9.65,7.8,43.799999
4,CA,52.799999,3.92,79.000000,7.28,9.14,9.9,38.799999
36,OR,48.900002,4.22,78.800003,6.91,8.88,9.5,36.500000


In [ ]:
df = LifeData().all_data
df = LifeData.filter_data_most_recent(LifeData,df = df)

In [ ]:
df

In [ ]:
pivot_state = df.pivot(index="State", columns="Indicator", values="Rate")
pivot_state = pivot_state.reset_index()
pivot_state

In [34]:
fig = px.histogram(pivot_state, y="Fertility Rate", marginal="violin",)
#fig.add_vline(x=60, line_color="red", line_width=3, line_dash="dash")
fig.update_layout(
    yaxis={'side': 'right'}  
)
fig.show()

In [35]:
fig = ff.create_distplot([pivot_state["Infant Mortality"]], ["Infant Mortality"])
fig.add_vline(x=6, line_color="red", line_width=3, line_dash="dash")
fig

In [50]:
fig = go.Figure()
fig.add_trace(go.Violin(y=pivot_state["Infant Mortality"], name="Infant Mortality", side="negative", meanline_visible=True))
fig.add_hline(y=6, line_color="red", line_width=3, line_dash="dash")
fig.show()


In [ ]:
merged_df.dtypes

In [ ]:
hist = ff.create_distplot([pivot_state["Infant Mortality"]], ["Infant Mortality"])
hist.add_vline(x=6, line_color="red", line_width=3, line_dash="dash")
hist.data[]

In [ ]:
fig = make_subplots(rows=3, cols=1, vertical_spacing= 0.01, subplot_titles=("Infant Mortality","","", ""))
hist = ff.create_distplot([pivot_state["Infant Mortality"]], ["Infant Mortality"])
fig.add_trace(hist.data[0], row=1, col=1)
fig.add_trace(hist.data[1], row=1, col=1)
#fig.add_trace(hist.data[2], row=1, col=1)
fig.add_trace(go.Scatter(y = merged_df["Low Birth Weight"], x = merged_df["Infant Mortality"], mode="markers",  
                         marker=dict(
                                size=6,
                                color=merged_df["Part_Score"], #set color equal to a variable
                                colorscale='Bluered', # one of plotly colorscales
                                #showscale=True
                        ),), row=2, col=1)
fig.add_trace(go.Scatter(y = merged_df["Premature"], 
                         x = merged_df["Infant Mortality"], 
                         mode="markers",  
                         marker=dict(
                                size=6,
                                color=merged_df["Part_Score"], #set color equal to a variable
                                colorscale='Bluered', # one of plotly colorscales
                                #showscale=True
                        ),), row=3, col=1)
#fig.add_trace(go.Histogram(x=pivot_state["Low Birth Weight"], nbinsx=15), row=1, col=2)
#fig.add_trace(go.Histogram(x=pivot_state["Premature"], histnorm="probability density", nbinsx=15), row=1, col=3)
fig.add_vline(x=6, line_color="green", line_width=3, line_dash="dash", row=1, col=1)
fig.add_vline(x=6, line_color="green", line_width=3, line_dash="dash", row=2, col=1)
fig.add_vline(x=6, line_color="green", line_width=3, line_dash="dash", row=3, col=1)
#fig.add_vline(x=6, line_color="green", line_width=3, line_dash="dash", row=4, col=1)
#Set xrange to 0-6
fig.update_xaxes(range=[3,10])
#remove xaxis from histogram
#fig.update_xaxes(showticklabels=False, row=1, col=1)
#fig.update_xaxes(showticklabels=False, row=2, col=1)
#fig.update_xaxes(showticklabels=False, row=3, col=1)
fig.update_layout(height=800, plot_bgcolor='rgba(0,0,0,0)')
fig['layout']['yaxis1']['title']=''
fig['layout']['yaxis1']['title']='Infant Mortality'
fig['layout']['yaxis2']['title']='Birth Weight'
fig['layout']['yaxis3']['title']='Premature'
fig['layout']['xaxis3']['title']='Infant Mortality'
fig.update_xaxes(showline=True, linewidth=2, linecolor='black', mirror=False)
fig.update_yaxes(showline=True, linewidth=2, linecolor='black', mirror=False)
#hide legend
fig.update_layout(showlegend=False)
#Add yaxis titles
fig.show()


In [ ]:
fig = make_subplots(rows=1, cols=2, subplot_titles=("Infant Mortality","Birth Weight", "Premature"))
hist = px.histogram(pivot_state, y="Infant Mortality", marginal="rug")
hist.update_layout(
    xaxis={'side': 'top'}, 
    yaxis={'side': 'right'}  
)
for i, t in enumerate(hist.data):
    fig.add_trace(t, row=1, col=2-i)
fig.show()

In [ ]:
pivot_state.columns

In [ ]:
merged_df.sample(3)

In [ ]:
fig = px.histogram(merged_df, x="Low Birth Weight", marginal="rug", color="category")
#fig.add_vline(x=60, line_color="red",  line_width=3, line_dash="dash")
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Box(
    y=merged_df["Infant Mortality"],
    marker_symbol='line-ns-open', 
    marker_color='blue',
    boxpoints='all',
    jitter=0,
    fillcolor='rgba(255,255,255,0)',
    line_color='rgba(255,255,255,0)',
    hoveron='points',
    name='rug'
))


In [ ]:
group_labels = ['distplot']
fig = ff.create_distplot([merged_df["Infant Mortality"]], group_labels )
fig.show()

In [ ]:
merged_df

In [ ]:
def create_birth_plots(state):
        state_infant_mortality = merged_df[merged_df["State"] == state]["Infant Mortality"].values[0]
        state_infant_premature = merged_df[merged_df["State"] == state]["Premature"].values[0]
        state_infant_birth = merged_df[merged_df["State"] == state]["Low Birth Weight"].values[0]
        state_conservativeness = merged_df[merged_df["State"] == state]["Part_Score"].values[0]
        
        

        
        
        fig = make_subplots(rows=4, cols=1, vertical_spacing= 0.01, subplot_titles=("Infant Mortality","","", ""))
        
        
        
        
        hist = ff.create_distplot([merged_df["Infant Mortality"]], ["Infant Mortality"])
        fig.add_trace(hist.data[0], row=1, col=1)
        fig.add_trace(hist.data[1], row=1, col=1)
        #fig.add_trace(hist.data[2], row=1, col=1)
        
        
        
        fig.add_trace(go.Scatter(y = merged_df["Low Birth Weight"], x = merged_df["Infant Mortality"], mode="markers",
                                 hovertext=merged_df["State"],    
                                 marker=dict(
                                        size=6,
                                        color=merged_df["Part_Score"], #set color equal to a variable
                                        colorscale='Bluered', # one of plotly colorscales
                                        #showscale=True
                                ),), row=2, col=1)
        
        fig.add_trace(go.Scatter(y = [state_infant_birth], x =[state_infant_mortality], mode="markers",  
                                 marker=dict(
                                        size=20,
                                        color="Green", 
                                        opacity=0.2,
                                ),), row=2, col=1)
        
        
        fig.add_trace(go.Scatter(y = merged_df["Premature"], 
                                 x = merged_df["Infant Mortality"], 
                                 mode="markers",
                                 hovertext=merged_df["State"],    
                                 marker=dict(
                                        size=6,
                                        color=merged_df["Part_Score"], #set color equal to a variable
                                        colorscale='Bluered', # one of plotly colorscales
                                        #showscale=True
                                ),), row=3, col=1)
        
        fig.add_trace(go.Scatter(y = [state_infant_premature], x =[state_infant_mortality], mode="markers",  
                                 marker=dict(
                                        size=20,
                                        color="Green", 
                                        opacity=0.2,
                                ),), row=3, col=1)
        
        #add scatter for Part_score
        fig.add_trace(go.Scatter(y = merged_df["Part_Score"], 
                                 x = merged_df["Infant Mortality"], 
                                 mode="markers",
                                 hovertext=merged_df["State"],  
                                 marker=dict(
                                        size=6,
                                        color=merged_df["Part_Score"], #set color equal to a variable
                                        colorscale='Bluered', # one of plotly colorscales
                                        #showscale=True
                                ),), row=4, col=1)
        
        fig.add_trace(go.Scatter(y = [state_conservativeness], 
                                 x =[state_infant_mortality], 
                                 mode="markers",  
                                 marker=dict(
                                        size=20,
                                        color="Green", 
                                        opacity=0.2,
                                ),
                                ), row=4, col=1)
        
        fig.add_vline(x=state_infant_mortality, line_color="green", line_width=2, line_dash="dash", row=1, col=1, opacity=0.7)
        fig.add_vline(x=state_infant_mortality, line_color="green", line_width=2, line_dash="dash", row=2, col=1, opacity=0.7)
        fig.add_vline(x=state_infant_mortality, line_color="green", line_width=2, line_dash="dash", row=3, col=1, opacity=0.7)
        fig.add_vline(x=state_infant_mortality, line_color="green", line_width=2, line_dash="dash", row=4, col=1, opacity=0.7)
        
        #update x range to min and max
        fig.update_xaxes(range=[math.floor(merged_df["Infant Mortality"].min()),math.ceil(merged_df["Infant Mortality"].max())])
        #remove xaxis from histogram
        fig.update_xaxes(showticklabels=False, row=1, col=1)
        fig.update_xaxes(showticklabels=False, row=2, col=1)
        fig.update_xaxes(showticklabels=False, row=3, col=1)
        fig.update_layout(height=800, plot_bgcolor='rgba(0,0,0,0)')
        fig['layout']['yaxis1']['title']=''
        fig['layout']['yaxis1']['title']='Infant Mortality State Dist'
        fig['layout']['yaxis2']['title']='Birth Weight/100k'
        fig['layout']['yaxis3']['title']='Premature Births/100k'
        fig['layout']['yaxis4']['title']='Conservativeness'
        fig['layout']['xaxis4']['title']='Infant Mortality/100k'
        fig.update_xaxes(showline=True, linewidth=2, linecolor='black', mirror=False)
        fig.update_yaxes(showline=True, linewidth=2, linecolor='black', mirror=False)
        #hide legend
        fig.update_layout(showlegend=False)
        #Add yaxis titles
        fig.show()
        

In [ ]:
create_birth_plots("AZ")

In [5]:
life_data = LifeData()
pivot_state =life_data.get_long_format()
part = Partisian.get_state_part_score()
df = pd.merge(part, pivot_state)

Checking for NAs in 2020 ind ['Life Exp'] rate 0
Checking for NAs in 2019 ind ['Life Exp'] rate 0
Checking for NAs in 2018 ind ['Life Exp'] rate 0
Checking for NAs in 2020 ind ['Infant Mortality'] rate 0
Checking for NAs in 2019 ind ['Infant Mortality'] rate 0
Checking for NAs in 2018 ind ['Infant Mortality'] rate 0
Checking for NAs in 2017 ind ['Infant Mortality'] rate 0
Checking for NAs in 2016 ind ['Infant Mortality'] rate 0
Checking for NAs in 2015 ind ['Infant Mortality'] rate 0
Checking for NAs in 2014 ind ['Infant Mortality'] rate 0
Checking for NAs in 2005 ind ['Infant Mortality'] rate 0
Checking for NAs in 2021 ind ['Fertility Rate'] rate 0
Checking for NAs in 2020 ind ['Fertility Rate'] rate 0
Checking for NAs in 2019 ind ['Fertility Rate'] rate 0
Checking for NAs in 2018 ind ['Fertility Rate'] rate 0
Checking for NAs in 2017 ind ['Fertility Rate'] rate 0
Checking for NAs in 2016 ind ['Fertility Rate'] rate 0
Checking for NAs in 2015 ind ['Fertility Rate'] rate 0
Checking for

In [10]:
df.sample(3)

,State,Part_Score,Year,category,Fertility Rate,Infant Mortality,Life Exp,Low Birth Weight,Premature,Teen Birth,Unmarried Birth
42,ID,1.000000,2024,Right,60.700001,5.06,78.400002,6.67,8.99,12.0,27.200001
11,MA,-0.533333,2024,Left,49.000000,3.94,79.000000,7.52,9.00,5.7,32.599998
27,AR,0.333333,2024,Right,61.700001,7.38,73.800003,9.52,12.04,26.5,46.299999


In [24]:
def get_best_fit(X, y):
    reg = LinearRegression().fit(X, y)
    x_range = np.linspace(X.min(), X.max())
    y = reg.predict(x_range.reshape(-1,1))
    return x_range, y[:,0], reg.coef_
new_x, new_y, coef = get_best_fit(df["Part_Score"].values.reshape(-1,1), df["Infant Mortality"].values.reshape(-1,1))

In [25]:
new_x

array([-1.        , -0.95918367, -0.91836735, -0.87755102, -0.83673469,
       -0.79591837, -0.75510204, -0.71428571, -0.67346939, -0.63265306,
       -0.59183673, -0.55102041, -0.51020408, -0.46938776, -0.42857143,
       -0.3877551 , -0.34693878, -0.30612245, -0.26530612, -0.2244898 ,
       -0.18367347, -0.14285714, -0.10204082, -0.06122449, -0.02040816,
        0.02040816,  0.06122449,  0.10204082,  0.14285714,  0.18367347,
        0.2244898 ,  0.26530612,  0.30612245,  0.34693878,  0.3877551 ,
        0.42857143,  0.46938776,  0.51020408,  0.55102041,  0.59183673,
        0.63265306,  0.67346939,  0.71428571,  0.75510204,  0.79591837,
        0.83673469,  0.87755102,  0.91836735,  0.95918367,  1.        ])

In [26]:
new_y

array([4.25530372, 4.29847926, 4.34165479, 4.38483033, 4.42800586,
       4.4711814 , 4.51435693, 4.55753247, 4.600708  , 4.64388354,
       4.68705907, 4.73023461, 4.77341014, 4.81658568, 4.85976121,
       4.90293675, 4.94611228, 4.98928782, 5.03246335, 5.07563889,
       5.11881442, 5.16198996, 5.20516549, 5.24834103, 5.29151656,
       5.3346921 , 5.37786763, 5.42104317, 5.4642187 , 5.50739424,
       5.55056977, 5.59374531, 5.63692084, 5.68009638, 5.72327191,
       5.76644745, 5.80962298, 5.85279852, 5.89597405, 5.93914959,
       5.98232512, 6.02550066, 6.06867619, 6.11185173, 6.15502726,
       6.1982028 , 6.24137833, 6.28455387, 6.3277294 , 6.37090494])

In [27]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = new_x, y = new_y)) 
fig.show()